Today, we're searching the **Ceres monitoring station** and we're helping adorable little Elf with their word search.

For the full details of the task, you can visit the official [Advent of Code 2024](https://adventofcode.com/2024/day/4) website.

And as per tradition, let's set the set the scene with ChatGPT illustration of the day.

<img src="./ai_illustrations/Day04.webp" width="50%" class="center" />

## Part 1: Searching for XMAS

We are given a letter grid, from which we need to find the word `XMAS`. However, the trick is that the word can be found in any direction: horizontally, vertically, or diagonally. Also, the word can be in reverse order.

For example, this would be a valid grid with all non-relevant letters replaced with `.`:

```
..X...
.SAMX.
.A..A.
XMAS.S
.X....
```

Our task is to find the number of times the word `XMAS` appears in the grid.

The example grid looks as follows:

```
MMMSXXMASM
MSAMXMSMSA
AMXSXMAAMM
MSAMASMSMX
XMASAMXAMM
XXAMMXXAMA
SMSMSASXSS
SAXAMASAAA
MAMMMXMMMM
MXMXAXMASX
```

There are **18 occurences** of the word `XMAS` in this grid. Here is the grid with all occurences highlighted:

```
....XXMAS.
.SAMXMS...
...S..A...
..A.A.MS.X
XMASAMX.MM
X.....XA.A
S.S.S.S.SS
.A.A.A.A.A
..M.M.M.MM
.X.X.XMASX
```

For this one, I will try to solve it in one pass. 

I will:  
* go through each character in the grid;   
* if I encounter an `X` or `S` I will mark down in the dictionary to check for next letters in the words at the possible directions (to the right, diagonal left, diagonal right and down) -- together with the expected letter I will also note down the direction of the word and the direction fo the spelling;  
* I will check if the poisition has been noted down before and if it has, I will check if the letter at this position is the one I expect; if so I will add the next position to the list of positions to check.  

This way, I will only traverse the grid once and I will not have to check the same position multiple times.

::: {.callout-note collapse="true" title="Setting up"}

In [1]:
from misc.helper import verify_answer

tiny_example = """
..X...
.SAMX.
.A..A.
XMAS.S
.X....
"""

tiny_example_answer = 4

example_input = """
MMMSXXMASM
MSAMXMSMSA
AMXSXMAAMM
MSAMASMSMX
XMASAMXAMM
XXAMMXXAMA
SMSMSASXSS
SAXAMASAAA
MAMMMXMMMM
MXMXAXMASX
"""

example_answer_p1 = 18

:::

In [2]:
from pathlib import Path


def part_one(input: str) -> int:
    xmas = "XMAS"

    # Load input if it's a file
    if Path(input).exists():
        with open(input) as f:
            input = f.read()

    check_position = {}
    count_words = 0
    # since I will be going through the grid from top to bottom, 
    # I will only need to mark for checking the positions to the right, down, 
    # down-right and down-left
    directions = [(0, 1), (1, -1), (1, 0), (1, 1)]

    for i, line in enumerate(input.strip().splitlines()):
        for j, char in enumerate(line):
            to_be_checked = check_position.pop((i, j), [])
            # if there are characters we are looking for, check if we found them
            for schar, coord, dir in to_be_checked:
                if schar == char:
                    if char in "XS":
                        count_words += 1
                    else:
                        next_pos = (i + coord[0], j + coord[1])
                        next_char = xmas[xmas.find(char) + dir]
                        check_position.setdefault(next_pos, []).append(
                            (next_char, coord, dir)
                        )

            # start a new search if char is "X" or "S"
            if char in "XS":
                direction_shift = 1 if char == "X" else -1
                for dx, dy in directions:
                    next_pos = (i + dx, j + dy)
                    next_char = xmas[xmas.find(char) + direction_shift]
                    check_position.setdefault(next_pos, []).append(
                        (next_char, (dx, dy), direction_shift)
                    )

        # clean up expired positions in the check_position dictionary
        check_position = {
            key: val for key, val in check_position.items() if key[0] > i
        }

    return count_words

In [3]:
verify_answer(part_one, tiny_example, tiny_example_answer)

✔️ That's right! The answer is 4.


In [4]:
verify_answer(part_one, example_input, example_answer_p1)

✔️ That's right! The answer is 18.


In [5]:
%time
part_one("./inputs/Day04.txt")

CPU times: user 4 μs, sys: 0 ns, total: 4 μs
Wall time: 9.06 μs


2560

> That's the right answer! You are one gold star ⭐ closer to finding the Chief Historian.

## Part 2: Searching for X-MAS 

In this task, we need to find the shape `X` created by words `MAS`, like this:

```
M.S
.A.
M.S
```

Taking the same example as in Part 1, but focusing on the `X-MAS` shape, we can see that there are **9 occurrences** of the `X-MAS` shape in the grid:

```
.M.S......
..A..MSMS.
.M.S.MAA..
..A.ASMSM.
.M.S.M....
..........
S.S.S.S.S.
.A.A.A.A..
M.M.M.M.M.
..........
```

The task is to find the number of times the `X-MAS` shape appears in the input.


This time around, I will not focus on traversing the input in one go -- but make this task simple for me. I will:

* read in the input to a matrix;  
* scan all submatrices of size 3x3.  

I will check each submatrix for the `X-MAS` shape, i.e.:  

* if the center is letter `A`,  
* if the upper right and lower left corners are `M` and `S`, or `S` and `M` respectively,
* and similarly if the upper left and lower right corners are `M` and `S`, or `S` and `M` respectively.

If all these conditions are met, I will increment the counter.

::: {.callout-note collapse="true" title="Saving test answer"}

In [6]:
example_answer_p2 = 9

:::

In [7]:
def read_input_to_matrix(input: str) -> list:
    if Path(input).exists():
        with open(input) as f:
            input = f.read()

    output = []

    for line in input.strip().splitlines():
        output.append([c for c in line.strip()])

    return output


def part_two(input: str) -> int:
    mat = read_input_to_matrix(input)

    count_mats = 0

    for i in range(len(mat[0]) - 2):
        for j in range(len(mat) - 2):
            center = mat[j + 1][i + 1]
            top_left, bottom_right = mat[j][i], mat[j + 2][i + 2]
            top_right, bottom_left = mat[j][i + 2], mat[j + 2][i]

            cond_center = center == "A"
            cond_lr = (top_left == "M" and bottom_right == "S") or (
                top_left == "S" and bottom_right == "M"
            )
            cond_rl = (top_right == "M" and bottom_left == "S") or (
                top_right == "S" and bottom_left == "M"
            )

            if cond_center and cond_lr and cond_rl:
                count_mats += 1

    return count_mats

In [8]:
verify_answer(part_two, example_input, example_answer_p2)

✔️ That's right! The answer is 9.


In [9]:
%time
part_two("./inputs/Day04.txt")

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 3.1 μs


1910

> That's the right answer! You are one gold star ⭐ closer to finding the Chief Historian.